##### 问题:
我们想在子类中扩展某个属性的功能，而这个属性是在父类中定义的。

##### 解决方案:
考虑如下的代码，这里我们定义了一个属性 name：

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
    
    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str): #如果传入的不是字符串,并报错
            raise TypeError('Expected a string')
        self._name = value #反之修改名字为value

    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

下面我们从 Person 类中继承，然后在子类中扩展 name 属性的功能：

In [2]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

下面是使用这个新类的示例：

In [3]:
s = SubPerson('Guido')

Setting name to Guido


In [4]:
s.name

Getting name


'Guido'

In [5]:
s.name = 'Larry'

Setting name to Larry


In [6]:
#s.name = 42

如果只想扩展属性中的其中一个方法，可以使用下面的代码实现：

In [7]:
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name 

或者，如果只想扩展 setter，可以这样：

In [8]:
class SubPerson(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

在子类中扩展属性会引入一些非常微妙的问题，因为属性其实是被定义为 getter、setter
和 deleter 方法的集合，而不仅仅只是单独的方法。因此，当我们扩展一个属性时，需
要弄清楚是要重新定义所有的方法还是只针对其中一个方法做扩展。

在第一个例子中，所有的属性方法都被重新定义了。在每个方法中，我们利用 super()
函数来调用之前的实现。在 setter 函数中，对 super(SubPerson, SubPerson).name.__
set__(self, value)的调用并不是错误，下面我们来解释一下。为了调用到 setter 之前的
实现，需要把控制流传递到之前定义的 name 属性的__set__()方法中去。但是，唯一
能调用到这个方法的方式就是以类变量而不是实例变量的方式去访问。这正是
<b>super(SubPerson, SubPerson)操作所完成的任务。

如果只想重新定义其中的一个方法，只使用@property 是不够的。例如，下面这样的代
码是无法工作的：

In [9]:
'''
class SubPerson(Person):
    @property # Doesn't work
    def name(self):
        print('Getting name')
        return super().name 
'''

"\nclass SubPerson(Person):\n    @property # Doesn't work\n    def name(self):\n        print('Getting name')\n        return super().name \n"

如果试着使用这份代码，就会发现 setter 函数完全消失不见了：

In [10]:
# s = SubPerson('Guido') 

相反，我们应该将代码修改为解决方案中的那样：

In [11]:
class SubPerson(Person):
    @Person.getter
    def name(self):
        print('Getting name')
        return super().name

AttributeError: type object 'Person' has no attribute 'getter'

当这么做之后，所有之前定义过的属性方法都会被拷贝过来，而 getter 函数则会被替换
掉。现在可以按照预期的方式工作了：

In [ ]:
s = SubPerson('Guido')
s.name

Setting name to Guido
Getting name


'Guido'